In [1]:
# import functions for model 
%run model_functions.ipynb
%run jude_plot_code.py

In [2]:
# import libraries
from __future__ import print_function, division
import sys
import numpy as np
import random as random
import math as math
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from random import triangular
import scipy.stats as sst
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import jude_plot_code as plot

In [3]:
# set code preferences for spiny dogfish model
year_split = 2008 # in what year should the data be split into estimation (below year_split) or validation (equals to or after year_split)? currently leaving 10 years for forecasting/validation
abc_options = ['regression','rejection']
abc_pref = abc_options[1] # choose which approach to Approximate Bayesian Computation to use 
temperature_options = ['ROMS','in_situ'] # ROMS is still the incorrect run from the cod forecast challenge
temperature_pref = temperature_options[1]
print(abc_pref) # check it's correct, and remember Python starts from 0! 
print(temperature_pref)

rejection
in_situ


In [4]:
# import spiny dogfish data & look at columns 
dat_trawl = pd.read_csv("../processed-data/dogfish_prepped_data.csv")
list(dat_trawl.columns)

['haulid',
 'lengthclass',
 'spp',
 'numlengthclass',
 'region',
 'year',
 'common',
 'stratum',
 'stratumarea',
 'lat',
 'lon',
 'depth',
 'btemp']

In [5]:
# keep only needed columns and import temperature data if needed 
if temperature_pref=='ROMS':
    dat_trawl = dat_trawl[['haulid','numlengthclass', 'year', 'lat','lengthclass']] # keep only needed columns 
    dat_roms = pd.read_csv("~/github/SDM-convergence/data/haul_ROMS.csv", usecols = ['unique_id',  'temp_bottom', 'temp_surface']) # import ROMS data 
    dat_roms.rename({"unique_id":"haulid"},axis="columns",inplace=True) # fix column names 
    dat_trawl = pd.merge(dat_estimation, dat_roms, how="inner", on="haulid") # merge with trawl data. because this is an inner join, it will omit NOAA hauls with no ROMS data, and ROMS data with no matches in the species' survey dataframe
if temperature_pref=='in_situ':
    dat_trawl = dat_trawl[['haulid','numlengthclass', 'year', 'lat','lengthclass','btemp']]
    dat_trawl.rename({'btemp':'temp_bottom'},axis="columns",inplace=True)
list(dat_trawl.columns) 

['haulid', 'numlengthclass', 'year', 'lat', 'lengthclass', 'temp_bottom']

In [6]:
# filter trawl data 
dat_estimation = dat_trawl.loc[(dat_trawl['year'] < year_split)] # use years before year_split for estimation

# check year filtering worked correctly
print(dat_trawl.year.min())
print(dat_trawl.year.max())

print(dat_estimation.year.min())
print(dat_estimation.year.max())

# round latitudes to integers
dat_estimation.lat = dat_estimation.lat.round().astype(np.int) # revisit and be more precise about rounding; currently rounding to nearest integer, so bands are defined as center points (35-degree band runs from 34.51 to 35.49)

1963
2018
1963
2007


/Users/afh/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [7]:
# taking care of missing data in both data frames
# AF: commenting this out because interpolation is a bit dodgy here. will just deal with NAs 
# dat_estimation=dat_estimation.interpolate(method ='linear', limit_direction ='forward')
# dat_estimation=dat_estimation.interpolate(method ='linear', limit_direction ='backward')
# dat_roms =dat_roms.interpolate(method ='linear', limit_direction ='forward')
# dat_roms=dat_roms.interpolate(method ='linear', limit_direction ='backward')


# USE DF = DF.DROPNA() INSTEAD

In [8]:
# track number of latitudes--currently the spatial unit of analysis
Nlat = dat_estimation['lat'].max()-dat_estimation['lat'].min()
latrange = np.arange(start=dat_estimation.lat.min(), stop=dat_estimation.lat.max(), step=1)
print(Nlat)
print(latrange)

18
[29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]


In [9]:
# here I'm temporarily reversing the changes to object names that I made 
df=dat_estimation
df.rename({"numlengthclass":"NUMLEN"},axis="columns",inplace=True)
#to track the total number of latitudes available
nn=df['lat'].max()-df['lat'].min()#15 #AF: add column that preserves true lat value, not just lat index 
#Libraries to keep track of the patches and stages # AF: make names more intuitive, consider using dataframes instead of dictionaries 
D={}
D1={}
# AF: what happens when all values are NA for mean() and sum() below? check that this isn't an issue

#extracting the data from the data frames  and storing according to the patch and stage. We start from the first to the last last patch (1:nn+1)  and when in each patch, we extract the number of species for  each life stage, the temperature for each patch (and compute the avaerage for the year)
for q in range(1,nn+2): # nn+2 because range() does not use the final value, i.e., range(1,3) equals [1,2]
    #Juveniles for patch 33+q( since the min patch is 34, we will start with patch 34 --to the maximum # AF: get rid of all fixed numerics here 
    D['J_patch'+ str(q)]=df.loc[(df['lat'] == (df['lat'].min()-1)+ q) & (df['lengthclass']=='smalljuv')]
    #total number of observations in each patch for each year
    n=len(D['J_patch'+ str(q)].year.values)
    # the total number of years of data available
    m=df['year'].max()-df['year'].min()
    #temperature readings when each species was caught in the patch
    Abun_TemJ=np.empty((m+1, 3))
    kJ=0
    kY=0
    kA=0
    for i in range (0, m+1):
        Abun_TemJ[i,0]=kJ
        DD=D['J_patch'+ str(q)]
        TT=DD.loc[(DD['year'] == 1980+i)]
        temp1=DD.loc[(DD['year'] == 1980+i)]
     #   print(temp1.temp_bottom.values) # AF: not sure why this is here 
        Abun_TemJ[i,1]=temp1.temp_bottom.values.mean()
        Abun_TemJ[i,2]=TT.NUMLEN.values.sum()
        
        kJ=kJ +1
    #After extracting teh temperature and calculating the mean value, we now save it
    D1['J_patch'+ str(q)]=Abun_TemJ
# now moving to Young Juveniles to perform teh same process as above
    D['Y_patch'+ str(q)]=df[(df['lat'] == 33+ q) & (df['lengthclass']=='largejuv')]
    n=len(D['Y_patch'+ str(q)].year.values)
    m=df['year'].max()-df['year'].min()
    Abun_TemY=np.empty((m+1, 3))
    for i in range (0, m+1):
        Abun_TemY[i,0]=kY
        DD=D['Y_patch'+ str(q)]
        TT=DD.loc[(DD['year'] == 1980+i)]
        temp1=DD.loc[(DD['year'] == 1980+i)]
        Abun_TemY[i,1]=temp1.temp_bottom.values.mean()
        Abun_TemY[i,2]=TT.NUMLEN.values.sum()
        kY=kY +1
    D1['Y_patch'+ str(q)]=Abun_TemY
#Next we move to Adult and perform teh same as above
    D['A_patch'+ str(q)]=df[(df['lat'] == 33+ q) & (df['lengthclass']=='adult')]
    Abun_TemA=np.empty((m+1, 3))
    for i in range (0, m+1):
        Abun_TemA[i,0]=kA
        DD=D['A_patch'+ str(q)]
        TT=DD.loc[(DD['year'] == 1980+i)]
        temp1=DD.loc[(DD['year'] == 1980+i)]
        Abun_TemA[i,1]=temp1.temp_bottom.values.mean()
        Abun_TemA[i,2]=TT.NUMLEN.values.sum()
        kA=kA +1
    D1['A_patch'+ str(q)]=Abun_TemA
    
    # use pd.ExcelWriter

/Users/afh/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/afh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: Mean of empty slice.
/Users/afh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: Mean of empty slice.
/Users/afh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Mean of empty slice.


In [ ]:
# now that data has been structured in a dictionary for the model, run the model
# we already imported the model functions at the top
#main script starts from here
# if __name__ == '__main__': # AF: ask Jude what this does -- apparently it denotes the main script--commenting out for now 
# Import the abundance data and data for the other variables e.g temperature
#  import groundfish_training AF: this isn't necessary anymore and I deleted "groundfish_training." from all the calls to D1 or D
# the total number of generations
T_FINAL = len(D1['J_patch1'][:,0])
#We simulate 20000 sets of parameters for for ABC, using non informatives priors (uniform priors
NUMBER_SIMS = 20000
#no of patches
no_patches=nn
rows=T_FINAL
cols=no_patches
# creating an array to store the number of juveniles, young juvenils and adults in each patch
N_J=np.ndarray(shape=(rows, cols), dtype=float, order='F') # rows are years and columns are patches 
N_Y=np.ndarray(shape=(rows, cols), dtype=float, order='F')
N_A=np.ndarray(shape=(rows, cols), dtype=float, order='F')
tempA = np.ndarray(shape=(rows, cols), dtype=float, order='F')
#storing data (secies abundance and temeprature time series data ) in the created arrays
for q in range(1,no_patches+1):
    i=q-1
    p=q
    N_J[:,i]=D1['J_patch'+ str(p)][:,2] # fill in the array with data from D1. column 2 in D1 holds abundance. column 0 contains indices (kJ, etc) and column 1 contains temperatures. 
    N_Y[:,i]=D1['Y_patch'+ str(p)][:,2]
    N_A[:,i]=D1['A_patch'+ str(p)][:,2]
    tempA[:,i]=D1['A_patch'+ str(p)][:,1] # only varies across patches, not life stages, so just need to save 1x 
#running ABC. See the function for details. returns all the observe summary statitics (OS)and simulated summary statistics (SS) in a matrix with first row corresponding to OS and the rest of the rows to SS as well as the parameter values that led to the simulated summary statistics.
param_save, Obs_Sim         = run_sim()

#normalize the rows of Obs_sim to have NOS in row 1 and NSS in the remaining rows. Substract rows i=2:NUMBER_SIMS from row 1 of Obs_sim (whic contain OS).Compute the eucleadean distance (d) between NSS and NOS then use it along side tolerance (δ), to determine all parameters and NSS corresponding to d ≤ δ.Choose δ such that δ × 100% of the NUMBER_SIMS simulated parameters and NSS are selected. retain the parameters that made this threshold (library), the weights ot be used in local linear regression and the NSS that meets the threshold (stats)
library, dists, stats,stats_SS,  NSS_cutoff, library_index   = sum_stats(Obs_Sim, param_save)
# performing rejectio ABC. Note that if UMBER_SIMS is big enough, but rejection and regression ABC leads to teh same results.
if abc_pref=='rejection':
    result, HPDR=do_rejection(library)
    print('see the results below')
    print('Estimates from rejection is:', result)
    print('Estimated HPDR from rejection is :', HPDR)
# Next we have regression ABC, perform it if only you are not performing rejection ABC above. Gives better results for NUMBER_SIMS small. I have commented it.
#  if abc_pref=='regression':
#      library_reg=do_logit_transformation(library, param_bound)LJ=34, Ly=68, Linf=200 # AFH: still commented out because the code is a little messed up
#      result_reg, HPDR_reg=do_kernel_ridge(stats, library_reg, param_bound)
PARAMS1={}
print(result[2]) # what are each of these? can we add a name column? 
PARAMS1 = {"L_0":result[0] , "L_inf": result[1],"L_J": 39.75,"L_Y": 67.5, "Topt": result[2], "width": result[3], "kopt": result[4],"xi":result[5], "m_J": result[6], "m_Y":result[7] , "m_A": result[8], "K": result[9]}

N_J1, N_Y1, N_A1 = simulation_population(PARAMS1) #AF: it appears that N_J1 contains the simulations for juveniles--all of them, not just patch 1? 


In [20]:
# trying to write out results of model to plot in R 
#print(type(N_J1))
#testdf = pd.DataFrame(N_J1)

#print(nn)
#print(testdf) # I think this has patch as columns and years as rows. would be nice to preserve the real year and patch IDs 

# Jude's code to write out to df 

print(PARAMS1)

yr=[]
lat=[]
stage=[]
abun=[]
for p in range(no_patches):
    for q in range(no_years):
        abun.append(N_J1[:,p][q])
        lat.append(latrange.min()+p) #lat.append(36+p)
        yr.append(year_split+q) #yr.append(2013+q)
        stage.append('smalljuv')
        abun.append(N_Y1[:,p][q])
        lat.append(latrange.min()+p) #lat.append(36+p)
        yr.append(year_split+q) #yr.append(2013+q)
        stage.append('largejuv')
        abun.append(N_A1[:,p][q])
        lat.append(latrange.min()+p) #lat.append(36+p)
        yr.append(year_split+q) #yr.append(2013+q)
        stage.append('adult')
df=pd.DataFrame({'Year': yr, 'Latitude':lat, 'State': stage, 'Abundance': abun})
print(df)

df.to_csv('spiny_dogfish_out.csv')

     Year  Latitude     State  Abundance
0    2008        29  smalljuv   0.000000
1    2008        29  largejuv   0.000000
2    2008        29     adult  20.000000
3    2009        29  smalljuv  20.106365
4    2009        29  largejuv   0.000000
..    ...       ...       ...        ...
535  2016        46  largejuv   0.000000
536  2016        46     adult   0.000000
537  2017        46  smalljuv   0.000000
538  2017        46  largejuv   0.000000
539  2017        46     adult   0.000000

[540 rows x 4 columns]


In [16]:
#Importing a file call plot to plot the results.
# print('i just imported a plot')
for q in range(1,no_patches+1):
    i=q-1
    p=q
    plot.do_realdata(N_J1[:,i], N_J[:,i],  'J_abun_rej'+ str(p))
    #plot.do_scatter(N_J1[:,i], N_J[:,i],  'J_abun_scatter'+ str(p))
    plot.do_realdata(N_Y1[:,i], N_Y[:,i],  'Y_abun_rej'+ str(p))
    #plot.do_scatter(N_Y1[:,i], N_Y[:,i],  'Y_abun_scatter'+ str(p))
    plot.do_realdata(N_A1[:,i], N_A[:,i],  'A_abun_rej'+ str(p))
#plot.do_scatter(N_A1[:,i], N_A[:,i],  'A_abun_scatter'+ str(p))
################################################################
# plot the figures below if you willl like to plot the heatmap
    NJ1=N_J1.transpose()
    NJ=N_J.transpose()
    NY1=N_Y1.transpose()
    NY=N_Y.transpose()
    NA1=N_A1.transpose()
    NA=N_A.transpose()
    print(NJ1.shape)
    ax=sns.heatmap(NJ1, cmap="Greys", xticklabels=True, yticklabels=True,  cbar_kws={'label': 'Abundance'})
    plt.xlabel("Year")
    plt.ylabel("Latitude")
    ax.set_xticklabels(pd.Series(range(1980, 2012)))
    ax.set_yticklabels(pd.Series(range(34, 46)))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.figure.savefig("sim_J.png", bbox_inches='tight')
    plt.close()
#############################################################
    ax = sns.heatmap(NJ, cmap="Greys",  xticklabels=True, yticklabels=True, cbar_kws={'label': 'Abundance'})
    plt.xlabel("Year")
    plt.ylabel("Latitude")
    ax.set_xticklabels(pd.Series(range(1980, 2012)))
    ax.set_yticklabels(pd.Series(range(34, 46)))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.figure.savefig("Obs_J.png", bbox_inches='tight')
    plt.close()
##########################################################
    ax = sns.heatmap(NY1, cmap="Greys", xticklabels=True, yticklabels=True,  cbar_kws={'label': 'Abundance'})
    plt.xlabel("Year")
    plt.ylabel("Latitude")
    ax.set_xticklabels(pd.Series(range(1980, 2013)))
    ax.set_yticklabels(pd.Series(range(34, 46)))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.figure.savefig("Sim_Y.png", bbox_inches='tight')
    plt.close()
##########################################################
    ax = sns.heatmap(NY, cmap="Greys", xticklabels=True, yticklabels=True,  cbar_kws={'label': 'Abundance'})
    plt.xlabel("Year")
    plt.ylabel("Latitude")
    ax.set_xticklabels(pd.Series(range(1980, 2013)))
    ax.set_yticklabels(pd.Series(range(34, 46)))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.figure.savefig("Obs_Y.png", bbox_inches='tight')
    plt.close()
############################################################
    ax = sns.heatmap(NA1, cmap="Greys", xticklabels=True, yticklabels=True, cbar_kws={'label': 'Abundance'})
    plt.xlabel("Year")
    plt.ylabel("Latitude")
    ax.set_xticklabels(pd.Series(range(1980, 2013)))
    ax.set_yticklabels(pd.Series(range(34, 46)))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.figure.savefig("Sim_A.png", bbox_inches='tight')
    plt.close()
############################################################
    ax = sns.heatmap(NA, cmap="Greys",  xticklabels=True, yticklabels=True, cbar_kws={'label': 'Abundance'})
    plt.xlabel("Year")
    plt.ylabel("Latitude")
    ax.set_xticklabels(pd.Series(range(1980, 2013)))
    ax.set_yticklabels(pd.Series(range(34, 46)))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.figure.savefig("Obs_A.png", bbox_inches='tight')
    plt.close()



ValueError: Length of passed values is 45, index implies 33.